In [132]:
# Import necessary libraries and modules
import os 
import re 
import requests 
from newspaper import Article 
from bs4 import BeautifulSoup 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.options import Options 
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains 
import time
import json
from tqdm import trange


In [133]:
options = Options()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)
driver.maximize_window()
base_url = "https://www.todayonline.com/"

In [134]:
# Want the article body, summary, and URL
scraped_articles = {}

In [135]:
# Function to navigate to a specific category URL
def go_to_category(driver, category_URL):
    driver.get(category_URL)

In [136]:
def scrape_categories(driver, base_url):
    driver.get(base_url)
    time.sleep(2)
        
    try:
        navbar_items = driver.find_elements(By.CSS_SELECTOR, "li.main-menu__item a.main-menu__link")
        exclude = ['watch, myfeed']
        sub_cats = []
        for item in navbar_items:
            heading = item.get_attribute("href").split('/')[-1]
            if heading not in exclude:
                sub_cats.append(heading)
    except Exception as e:
        print(f"Error extracting sub category: {e}")
    sub_cats.remove('watch')
    sub_cats.remove('myfeed')
    return sub_cats


In [137]:
def scrape_categories(driver, base_url):
    driver.get(base_url)
    time.sleep(2)
        
    try:
        navbar_items = driver.find_elements(By.CSS_SELECTOR, "li.main-menu__item a.main-menu__link")
        exclude = ['watch, myfeed']
        sub_cats = []
        for item in navbar_items:
            heading = item.get_attribute("href").split('/')[-1]
            if heading not in exclude:
                sub_cats.append(heading)
    except Exception as e:
        print(f"Error extracting sub category: {e}")
    sub_cats.remove('watch')
    sub_cats.remove('myfeed')
    return sub_cats

# Function to gather article URLs from a category page
def gather_article_urls(driver):
    urls = []
    article_cards = driver.find_elements(By.CLASS_NAME, "card-object__figure")
    for card in article_cards:
        try:
            article_link = card.find_element(By.CLASS_NAME, "link")
            url = article_link.get_attribute("href")
            urls.append(url)
        except Exception as e:
            print(f"Error finding article URL: {e}")
    return urls

# Function to scrape articles from gathered URLs
def scrape_articles(driver, urls, sub_url):
    sub_url_ls = []
    for i in trange(len(urls)):
        sub_url_hash = {}
        sub_url_hash["Article URL"] = urls[i]
        try:
                # Navigate to the article page
                driver.get(urls[i])
                time.sleep(2)  # Waiting for the page to load

                # Get article summaries
                summary_elements = driver.find_elements(By.CSS_SELECTOR, "div.text-long ul li")
                summary_texts = [li.text for li in summary_elements]
                Combined_Summary = ".\n".join(summary_texts)
                if len(Combined_Summary) == 0:
                    sub_url_hash["Article Summary"] = 'NO SUMMARY'
                else:
                    sub_url_hash["Article Summary"] = Combined_Summary

                # Get article body
                try:
                    paragraphs = driver.find_elements(By.CSS_SELECTOR, "div.text-long p")
                    body = "\n".join([p.text for p in paragraphs])
                    sub_url_hash['Article Body'] = body
                except Exception as e:
                    sub_url_hash['Article Body'] = None
                    print("Body paragraphs not extracted")

        except Exception as e:
                print(f"Error scraping article: {e}")
        sub_url_ls.append(sub_url_hash)
        if i >2:
            break
    scraped_articles[sub_url] = sub_url_ls

In [138]:
driver.get(base_url)
time.sleep(2)  # Wait for the page to load

category_sub_URLs = scrape_categories(driver, base_url)

In [139]:
for sub_url in category_sub_URLs:
    full_category_url = f"{base_url}{sub_url}"
    go_to_category(driver, full_category_url)
    urls = gather_article_urls(driver)
    scrape_articles(driver, urls, sub_url)
    break 
    
# Close the driver after scraping
driver.quit()

 12%|█▏        | 3/26 [00:14<01:48,  4.70s/it]


In [140]:
scraped_articles

{'news': [{'Article URL': 'https://www.todayonline.com/news/2-men-arrested-fight-dangerous-weapon-tiong-bahru-2431576',
   'Article Summary': 'NO SUMMARY',
   'Article Body': 'SINGAPORE — Two men were arrested after they were seen engaging in a fight in Tiong Bahru on Tuesday (May 28), with a screwdriver being seized by the police.\nThe roadside altercation was captured in an online video. It is unclear what caused the fight. \nThe police told TODAY on Wednesday that they received a call for assistance at Block 54 Havelock Road at 12.05pm on Tuesday.\nTwo men, aged 38 and 62, were arrested for affray. The older man was arrested for voluntarily causing hurt by a dangerous weapon as well. \nBoth sustained minor injuries but refused to be taken to the hospital. \nAffray is fighting in public in a way that disturbs the peace.\nPolice investigations are ongoing. \nIn a 43-second video published on Facebook group SG Road Vigilante, the 62-year-old man in a white shirt was seen charging at th

In [141]:
with open("today_online/articles.json", "w", encoding='utf-8') as fout:
    json.dump(scraped_articles, fout, indent=4, ensure_ascii=False)